In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import joblib
from sklearn.ensemble import RandomForestClassifier
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split, KFold
import sys
import os
from os import listdir
from os.path import isfile, join
import pandas as pd
from deslib.util.instance_hardness import kdn_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.naive_bayes import BernoulliNB, GaussianNB
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.datasets import make_classification
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, roc_auc_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix
from tabulate import tabulate
from tqdm import tqdm
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier
from deslib.dcs import OLA
# from deslib.dcs import KNORA_U, KNORA_E
from deslib.des import  KNOP, METADES
from deslib.static import SingleBest, StackedClassifier
from deslib.static import StaticSelection
from imblearn.metrics import geometric_mean_score
# from pyhard import HardMetrics

In [ ]:
FEATURES_SET = {
    "feature": 1,
    "permission": 2,
    "activity": 3,
    "service_receiver": 3,
    "provider": 3,
    "service": 3,
    "intent": 4,
    "api_call": 5,
    "real_permission": 6,
    "call": 7,
    "url": 8
}


def count_feature_set(lines):
    """
    Count how many features belong to a specific set
    :param lines: features in the text file
    :return:
    """
    features_map = {x: 0 for x in range(1, 9)}
    for l in lines:
        if l != "\n":
            set = l.split("::")[0]
            features_map[FEATURES_SET[set]] += 1
    features = []
    for i in range(1, 9):
        features.append(features_map[i])
    return features


In [ ]:
def read(LOAD_DATA=False):
    if LOAD_DATA:
        print("Previous data not loaded. Attempt to read data ...")
        mypath = r"Drebin\MetaData\feature_vectors\feature_vectors"
        onlyfiles = [f for f in listdir(mypath) if isfile(join(mypath, f))]

        print("Reading csv file for ground truth ...")
        ground_truth = np.loadtxt(r"Drebin\MetaData\sha256_family.csv", delimiter=",", skiprows=1, dtype=str)
        # print ground_truth.shape
        # families = np.unique(ground_truth[:, 1])
        # print families
        # print len(families)

        print("Reading positive and negative texts ...")
        pos = []
        neg = []
        for virus in tqdm(onlyfiles):
            if virus in ground_truth[:, 0]:
                pos.append(virus)
            else:
                neg.append(virus)

        print("Extracting features ...")
        x = []
        y = []
        for text_file in tqdm(pos):
            sys.stdin = open("%s/%s" % (mypath, text_file))
            features = sys.stdin.readlines()
            sample = count_feature_set(features)
            x.append(sample)
            y.append(1)

        for text_file in tqdm(neg):
            sys.stdin = open("%s/%s" % (mypath, text_file))
            features = sys.stdin.readlines()
            sample = count_feature_set(features)
            x.append(sample)
            y.append(0)

        print("Data is read successfully:")
        x = np.array(x)
        y = np.array(y)
        print(x.shape, y.shape)

        print("Saving data under data_numpy directory ...")
        np.save(r"x_all.npy", x)
        np.save(r"y_all.npy", y)

        return x, y
    else:
        print("Loading previous data ...")
        x_ = np.load(r"x_all.npy")
        y_ = np.load(r"y_all.npy")
        print(x_.shape, y_.shape)
        # print x == x_, y == y_
        return x_, y_


def map_family_to_category(families):
    out = {}
    count = 1
    for family in families:
        out[family] = count
        count += 1
    return out


if __name__ == "__main__":
    #x, y = read(LOAD_DATA=True)
    x, y = read()

In [ ]:
import os
import joblib
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

models = {}

# Função para ler os dados
x0, y0 = read(LOAD_DATA=False)

# Caminho do diretório com os modelos
dir_path = r'E:\DrebinStudy\Balanced\Bagging2'

# Função para criar o gráfico cumulativo para uma seed específica
def plot_cumulative_histogram_single_seed(hardness_table, seed):
    plt.figure(figsize=(10, 6))

    for label in ['label_0', 'label_1']:    
        # Hits para a seed especificada
        hit_scores = hardness_table[label]['Hit'][seed]
        sorted_hits = np.sort(hit_scores)
        cumulative_hits = np.arange(1, len(sorted_hits) + 1) / len(sorted_hits)
        plt.plot(sorted_hits, cumulative_hits, label=f"{label} - Hit")

        # Misses para a seed especificada
        miss_scores = hardness_table[label]['Miss'][seed]
        sorted_misses = np.sort(miss_scores)
        cumulative_misses = np.arange(1, len(sorted_misses) + 1) / len(sorted_misses)
        plt.plot(sorted_misses, cumulative_misses, label=f"{label} - Miss", linestyle='--')

    # Configurações do gráfico
    plt.title(f'Cumulative Hardness Histogram for {seed}')
    plt.xlabel('Hardness Score')
    plt.ylabel('Cumulative Probability')
    plt.legend()
    plt.grid(True)
    plt.show()

# Loop para percorrer todas as subpastas dentro de 'Bagging2'
for subfolder_name in os.listdir(dir_path):
    subfolder_path = os.path.join(dir_path, subfolder_name)
    
    # Verifica se é uma subpasta
    if os.path.isdir(subfolder_path):
        print(f'Processando subpasta: {subfolder_name}')
        i = 0
        hardness_table = {'label_0': {'Hit': {}, 'Miss': {}}, 'label_1': {'Hit': {}, 'Miss': {}}}

        # Loop pelas 30 pastas dentro de cada subpasta (1 a 30)
        for model_folder in os.listdir(subfolder_path):
            model_folder_path = os.path.join(subfolder_path, model_folder)
            
            if os.path.isdir(model_folder_path):
                # Carregar todos os modelos na pasta atual
                models_list = []
                for model_file in os.listdir(model_folder_path):
                    model_path = os.path.join(model_folder_path, model_file)
                    model = joblib.load(model_path)
                    models_list.append(model)

                # Fazer previsões combinadas para cada modelo
                x_train, x_test, y_train, y_test = train_test_split(x0, y0, test_size=0.2, stratify=y0, random_state=i)
                y_preds = np.array([model.predict(x_test) for model in models_list])
                previsoes = np.apply_along_axis(lambda x: np.bincount(x).argmax(), axis=0, arr=y_preds)

                # Calcular o hardness score
                hardness_score = kdn_score(x_test, y_test, k=7)[0]

                # Separar os scores por label
                for label in [0, 1]:
                    hit_scores = hardness_score[(y_test == previsoes) & (y_test == label)]
                    miss_scores = hardness_score[(y_test != previsoes) & (y_test == label)]
                    hardness_table[f'label_{label}']['Hit'][f'seed-{str(i)}'] = hit_scores
                    hardness_table[f'label_{label}']['Miss'][f'seed-{str(i)}'] = miss_scores

                print(f"Hardness médio para seed-{i}: {np.mean(hardness_score):.4f}")
                i += 1
                print(f'Modelos processados na pasta: {model_folder_path}')
                
                # Gerar gráfico cumulativo para o modelo atual
                plot_cumulative_histogram_single_seed(hardness_table, f'seed-{i-1}')  # Mostrar para o último seed processado

        # Salvar a hardness_table por subpasta
        models[subfolder_name] = hardness_table


In [ ]:
import plotly.express as px
import pandas as pd
from sklearn.manifold import TSNE
from sklearn.preprocessing import StandardScaler

# Opcional: normalizar os dados
scaler = StandardScaler()
x_scaled = scaler.fit_transform(x)

# Aplicar t-SNE ou UMAP
tsne = TSNE(n_components=3, random_state=42)
x_embedded = tsne.fit_transform(x_scaled)

# Colocar o resultado em um DataFrame para visualização
df = pd.DataFrame(x_embedded, columns=['Dim1', 'Dim2', 'Dim3'])
df['Label'] = y  # Adicionar rótulos

# Criar o gráfico 3D interativo
fig = px.scatter_3d(df, x='Dim1', y='Dim2', z='Dim3', color='Label',
                    title="Visualização do Espaço de Instâncias com t-SNE",
                    labels={"Dim1": "Componente 1", "Dim2": "Componente 2", "Dim3": "Componente 3"})

# Exibir o gráfico
fig.show()


In [ ]:
smote = SMOTE(sampling_strategy='auto', k_neighbors=5)
x, y = smote.fit_resample(x, y)

In [ ]:
import plotly.express as px
import pandas as pd
from sklearn.manifold import TSNE
from sklearn.preprocessing import StandardScaler

# Opcional: normalizar os dados
scaler = StandardScaler()
x_scaled = scaler.fit_transform(x)

# Aplicar t-SNE ou UMAP
tsne = TSNE(n_components=3, random_state=42)
x_embedded = tsne.fit_transform(x_scaled)

# Colocar o resultado em um DataFrame para visualização
df = pd.DataFrame(x_embedded, columns=['Dim1', 'Dim2', 'Dim3'])
df['Label'] = y  # Adicionar rótulos

# Criar o gráfico 3D interativo
fig = px.scatter_3d(df, x='Dim1', y='Dim2', z='Dim3', color='Label',
                    title="Visualização do Espaço de Instâncias com t-SNE",
                    labels={"Dim1": "Componente 1", "Dim2": "Componente 2", "Dim3": "Componente 3"})

# Exibir o gráfico
fig.show()

In [ ]:
x0, y0 = read(LOAD_DATA=False)

In [ ]:
# Identificando as novas instâncias
num_original_samples = len(x0)
num_resampled_samples = len(x)

# Adicionando a classe 2 às novas instâncias
new_labels = np.array([2] * (num_resampled_samples - num_original_samples))

# Unindo os rótulos gerados (classe 2) ao dataset balanceado
# As classes 0 e 1 continuam com seus rótulos originais
y_modified = np.concatenate([y0, new_labels])


In [ ]:
import plotly.express as px
import pandas as pd
from sklearn.manifold import TSNE
from sklearn.preprocessing import StandardScaler

# Opcional: normalizar os dados
scaler = StandardScaler()
x_scaled = scaler.fit_transform(x)

# Aplicar t-SNE ou UMAP
tsne = TSNE(n_components=3, random_state=42)
x_embedded = tsne.fit_transform(x_scaled)

# Colocar o resultado em um DataFrame para visualização
df = pd.DataFrame(x_embedded, columns=['Dim1', 'Dim2', 'Dim3'])
df['Label'] = y_modified  # Adicionar rótulos

# Criar o gráfico 3D interativo
fig = px.scatter_3d(df, x='Dim1', y='Dim2', z='Dim3', color='Label',
                    title="Visualização do Espaço de Instâncias com t-SNE",
                    labels={"Dim1": "Componente 1", "Dim2": "Componente 2", "Dim3": "Componente 3"})

# Exibir o gráfico
fig.show()

In [ ]:
hardness_table = {}
x0, y0 = read(LOAD_DATA=False)
for i in range(30):
    x_train, x_test, y_train, y_test = train_test_split(x0, y0, test_size=0.2, stratify=y0, random_state = i)
    hardness_score = kdn_score(x_test, y_test, k=7)[0]
    hardness_table[f"seed-{i}"] = hardness_score
    print(f"Hardness médio para {f'seed-{i}'}: {np.mean(hardness_score):.4f}")

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Número de gráficos que você deseja (5 neste caso)
num_plots = 6

# Criar subplots (uma figura com múltiplos gráficos)
fig, axs = plt.subplots(num_plots, 1, figsize=(8, num_plots * 4))  # 5 gráficos verticais

# Definir a quantidade de modelos por gráfico
models_per_plot = len(hardness_table) // num_plots

# Iterador para controlar qual gráfico está sendo plotado
plot_idx = 0

# Iterar sobre os modelos
for i, model_name in enumerate(hardness_table):
    # Obter dados ordenados e calcular cumulativo
    sorted_data = np.sort(hardness_table[model_name])
    cumulative = np.arange(1, len(sorted_data) + 1) / len(sorted_data)
    
    # Plotar no gráfico atual
    axs[plot_idx].plot(sorted_data, cumulative, label=f'{model_name}')
    axs[plot_idx].set_title(f"Hardness Distribution for Models {plot_idx+1}")
    axs[plot_idx].set_xlabel("Hardness")
    axs[plot_idx].set_ylabel("Cumulative Percentage")
    axs[plot_idx].set_ylim(0.90, 1.0)
    axs[plot_idx].grid(True)
    axs[plot_idx].legend()
    
    # Mudar para o próximo gráfico a cada `models_per_plot` modelos
    if (i + 1) % models_per_plot == 0 and plot_idx < num_plots - 1:
        plot_idx += 1

# Ajustar o layout para evitar sobreposição
plt.tight_layout()

# Mostrar o gráfico
plt.show()


In [ ]:
hardness_table = {'label_0': [], 'label_1': []}

# Ler os dados
x0, y0 = read(LOAD_DATA=False)

for i in range(30):
    x_train, x_test, y_train, y_test = train_test_split(x0, y0, test_size=0.2, stratify=y0, random_state=i)
    hardness_score = kdn_score(x_test, y_test, k=7)[0]
    
    # Separar os scores por label
    for label in [0, 1]:
        label_scores = hardness_score[y_test == label]
        hardness_table[f'label_{label}'].extend(label_scores)
    
    print(f"Hardness médio para seed-{i}: {np.mean(hardness_score):.4f}")

# Número de gráficos que você deseja
num_plots = 2

# Criar subplots (uma figura com múltiplos gráficos)
fig, axs = plt.subplots(num_plots, 1, figsize=(8, num_plots * 4))

# Iterar sobre os labels
for idx, label in enumerate([0, 1]):
    # Obter dados ordenados e calcular cumulativo
    sorted_data = np.sort(hardness_table[f'label_{label}'])
    cumulative = np.arange(1, len(sorted_data) + 1) / len(sorted_data)
    
    # Plotar no gráfico atual
    axs[idx].plot(sorted_data, cumulative, label=f'Label {label}')
    axs[idx].set_title(f"Hardness Distribution for Label {label}")
    axs[idx].set_xlabel("Hardness")
    axs[idx].set_ylabel("Cumulative Percentage")
    axs[idx].set_ylim(0.0, 1.0)
    axs[idx].grid(True)
    axs[idx].legend()

# Ajustar o layout para evitar sobreposição
plt.tight_layout()

# Mostrar o gráfico
plt.show()

In [ ]:
hardness_table = {'label_0': {}, 'label_1': {}}

# Ler os dados
x0, y0 = read(LOAD_DATA=False)

for i in range(30):
    x_train, x_test, y_train, y_test = train_test_split(x0, y0, test_size=0.2, stratify=y0, random_state=i)
    hardness_score = kdn_score(x_test, y_test, k=7)[0]
    
    # Separar os scores por label
    for label in [0, 1]:
        label_scores = hardness_score[y_test == label]
        hardness_table[f'label_{label}'][f'seed-{str(i)}'] = label_scores
    
    print(f"Hardness médio para seed-{i}: {np.mean(hardness_score):.4f}")

# Número de gráficos que você deseja
num_plots = 6

# Iterar sobre os modelos
for i, model_name in enumerate(hardness_table):
    # Obter dados ordenados e calcular cumulativo
    sorted_data = np.sort(hardness_table[model_name])
    cumulative = np.arange(1, len(sorted_data) + 1) / len(sorted_data)
    
    # Plotar no gráfico atual
    axs[plot_idx].plot(sorted_data, cumulative, label=f'{model_name}')
    axs[plot_idx].set_title(f"Hardness Distribution for Models {plot_idx+1}")
    axs[plot_idx].set_xlabel("Hardness")
    axs[plot_idx].set_ylabel("Cumulative Percentage")
    axs[plot_idx].set_ylim(0.90, 1.0)
    axs[plot_idx].grid(True)
    axs[plot_idx].legend()
    
    # Mudar para o próximo gráfico a cada `models_per_plot` modelos
    if (i + 1) % models_per_plot == 0 and plot_idx < num_plots - 1:
        plot_idx += 1

# Ajustar o layout para evitar sobreposição
plt.tight_layout()

# Mostrar o gráfico
plt.show()

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

def plot_cumulative_hardness_scores(hardness_table):
    # Plotar gráficos cumulativos para cada seed e label
    for label in hardness_table.keys():
        plt.figure(figsize=(12, 6))
        
        for seed in hardness_table[label].keys():
            scores = np.array(hardness_table[label][seed])
            # Ordenar scores para criar o gráfico cumulativo
            sorted_scores = np.sort(scores)
            # Calcular a distribuição cumulativa
            cumulative_distribution = np.arange(1, len(sorted_scores) + 1) / len(sorted_scores)
            plt.plot(sorted_scores, cumulative_distribution, label=f'Seed {seed}')
        
        plt.title(f'Cumulative Hardness Scores for Label {label}')
        plt.xlabel('Hardness Score')
        plt.ylabel('Cumulative Probability')
        plt.legend()
        plt.grid(True)
        plt.show()

    # Plotar gráficos cumulativos agregados para cada label
    for label in hardness_table.keys():
        plt.figure(figsize=(12, 6))
        all_scores = np.concatenate([np.array(scores) for scores in hardness_table[label].values()])
        sorted_scores = np.sort(all_scores)
        cumulative_distribution = np.arange(1, len(sorted_scores) + 1) / len(sorted_scores)
        plt.plot(sorted_scores, cumulative_distribution, label=f'Label {label}')
        
        plt.title(f'Cumulative Hardness Scores for Label {label}')
        plt.xlabel('Hardness Score')
        plt.ylabel('Cumulative Probability')
        plt.legend()
        plt.grid(True)
        plt.show()

# Chamada da função
plot_cumulative_hardness_scores(hardness_table)


In [ ]:
import matplotlib.pyplot as plt
import numpy as np

def plot_cumulative_hardness_scores_for_seed(hardness_table, seed):
    plt.figure(figsize=(12, 6))
    
    # Plotar gráficos cumulativos para cada label
    for label in hardness_table.keys():
        scores = np.array(hardness_table[label][seed])
        # Ordenar scores para criar o gráfico cumulativo
        sorted_scores = np.sort(scores)
        # Calcular a distribuição cumulativa
        cumulative_distribution = np.arange(1, len(sorted_scores) + 1) / len(sorted_scores)
        plt.plot(sorted_scores, cumulative_distribution, label=f'Label {label}')
    
    plt.title(f'Cumulative Hardness Scores for Seed {seed}')
    plt.xlabel('Hardness Score')
    plt.ylabel('Cumulative Probability')
    plt.legend()
    plt.grid(True)
    plt.show()

# Escolher a seed específica que você deseja plotar
seed_to_plot = 'seed-0'

# Chamada da função para a seed específica
plot_cumulative_hardness_scores_for_seed(hardness_table, seed_to_plot)


In [ ]:
import os

models = {}

hardness_table = {'label_0': {'Hit':{}, 'Miss':{}}, 'label_1': {'Hit':{}, 'Miss':{}}}

# Ler os dados
x0, y0 = read(LOAD_DATA=False)

# Caminho do diretório com os modelos
dir_path = r'E:\DrebinStudy\Unbalanced'

# Loop para percorrer todas as pastas no diretório
for folder_name in os.listdir(dir_path)[:1]:
    folder_path = os.path.join(dir_path, folder_name)
    
    # Verifica se é uma pasta e se não é a pasta 'Metrics'
    if os.path.isdir(folder_path) and folder_name != 'Metrics':
        print(f'Processando pasta: {folder_name}')
        i = 0
        hardness_table = {'label_0': {'Hit':{}, 'Miss':{}}, 'label_1': {'Hit':{}, 'Miss':{}}}
        # Aqui você pode inserir o código para processar os modelos dentro da pasta
        for file_name in os.listdir(folder_path):
            file_path = os.path.join(folder_path, file_name)
            x_train, x_test, y_train, y_test = train_test_split(x0, y0, test_size=0.2, stratify=y0, random_state=i)
            hardness_score = kdn_score(x_test, y_test, k=7)[0]
            model = joblib.load(file_path)
            y_pred = model.predict(x_test)

            # Separar os scores por label
            for label in [0, 1]:
                hit_scores = hardness_score[(y_test == y_pred) & (y_test == label)]
                miss_scores = hardness_score[(y_test != y_pred) & (y_test == label)]
                hardness_table[f'label_{label}']['Hit'][f'seed-{str(i)}'] = hit_scores
                hardness_table[f'label_{label}']['Miss'][f'seed-{str(i)}'] = miss_scores
            
            print(f"Hardness médio para seed-{i}: {np.mean(hardness_score):.4f}")
            i += 1   
            # Faz algo com os arquivos de modelo
            print(f'Arquivo de modelo encontrado: {file_path}')
        models[folder_name] = hardness_table

In [ ]:
import os
import joblib
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

models = {}

# Função para ler os dados
x0, y0 = read(LOAD_DATA=False)

# Caminho do diretório com os modelos
dir_path = r'E:\DrebinStudy\Unbalanced'

# Função para criar o gráfico cumulativo para uma seed específica
def plot_cumulative_histogram_single_seed(hardness_table, seed):
    plt.figure(figsize=(10, 6))

    for label in ['label_0', 'label_1']:
        # Hits para a seed especificada
        hit_scores = hardness_table[label]['Hit'][seed]
        sorted_hits = np.sort(hit_scores)
        cumulative_hits = np.arange(1, len(sorted_hits) + 1) / len(sorted_hits)
        plt.plot(sorted_hits, cumulative_hits, label=f"{label} - Hit")

        # Misses para a seed especificada
        miss_scores = hardness_table[label]['Miss'][seed]
        sorted_misses = np.sort(miss_scores)
        cumulative_misses = np.arange(1, len(sorted_misses) + 1) / len(sorted_misses)
        plt.plot(sorted_misses, cumulative_misses, label=f"{label} - Miss", linestyle='--')

    # Configurações do gráfico
    plt.title(f'Cumulative Hardness Histogram for {seed}')
    plt.xlabel('Hardness Score')
    plt.ylabel('Cumulative Probability')
    plt.legend()
    plt.grid(True)
    plt.show()

# Loop para percorrer todas as pastas no diretório (tirando 'Metrics')
for folder_name in os.listdir(dir_path):
    folder_path = os.path.join(dir_path, folder_name)
    
    # Verifica se é uma pasta e se não é a pasta 'Metrics'
    if os.path.isdir(folder_path) and folder_name != 'Metrics':
        print(f'Processando pasta: {folder_name}')
        i = 0
        hardness_table = {'label_0': {'Hit':{}, 'Miss':{}}, 'label_1': {'Hit':{}, 'Miss':{}}}

        # Processar os modelos dentro da pasta
        for file_name in os.listdir(folder_path):
            file_path = os.path.join(folder_path, file_name)
            x_train, x_test, y_train, y_test = train_test_split(x0, y0, test_size=0.2, stratify=y0, random_state=i)
            hardness_score = kdn_score(x_test, y_test, k=7)[0]
            model = joblib.load(file_path)
            y_pred = model.predict(x_test)

            # Separar os scores por label
            for label in [0, 1]:
                hit_scores = hardness_score[(y_test == y_pred) & (y_test == label)]
                miss_scores = hardness_score[(y_test != y_pred) & (y_test == label)]
                hardness_table[f'label_{label}']['Hit'][f'seed-{str(i)}'] = hit_scores
                hardness_table[f'label_{label}']['Miss'][f'seed-{str(i)}'] = miss_scores

            print(f"Hardness médio para seed-{i}: {np.mean(hardness_score):.4f}")
            i += 1   
            print(f'Arquivo de modelo encontrado: {file_path}')
            
            # Gerar gráfico cumulativo para o modelo atual
            plot_cumulative_histogram_single_seed(hardness_table, f'seed-{i-1}')  # Mostrar para o último seed processado
        
        # Salvar a hardness_table por pasta
        models[folder_name] = hardness_table

In [ ]:
hardness_table = {'label_0': {'Hit':{}, 'Miss':{}}, 'label_1': {'Hit':{}, 'Miss':{}}}

# Ler os dados
x0, y0 = read(LOAD_DATA=False)

for i in range(30):
    x_train, x_test, y_train, y_test = train_test_split(x0, y0, test_size=0.2, stratify=y0, random_state=i)
    hardness_score = kdn_score(x_test, y_test, k=7)[0]
    model = joblib.load(f"E:\DrebinStudy\Balanced\RandomForest\model_{i+1}.joblib")
    y_pred = model.predict(x_test)

    # Separar os scores por label
    for label in [0, 1]:
        hit_scores = hardness_score[(y_test == y_pred) & (y_test == label)]
        miss_scores = hardness_score[(y_test != y_pred) & (y_test == label)]
        hardness_table[f'label_{label}']['Hit'][f'seed-{str(i)}'] = hit_scores
        hardness_table[f'label_{label}']['Miss'][f'seed-{str(i)}'] = miss_scores
    
    print(f"Hardness médio para seed-{i}: {np.mean(hardness_score):.4f}")

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Função para criar o gráfico cumulativo
def plot_cumulative_histogram(hardness_table):
    plt.figure(figsize=(10, 6))

    for label in ['label_0', 'label_1']:
        # Hits
        hit_scores = np.concatenate([hardness_table[label]['Hit'][seed] for seed in hardness_table[label]['Hit']])
        sorted_hits = np.sort(hit_scores)
        cumulative_hits = np.arange(1, len(sorted_hits) + 1) / len(sorted_hits)
        plt.plot(sorted_hits, cumulative_hits, label=f"{label} - Hit")

        # Misses
        miss_scores = np.concatenate([hardness_table[label]['Miss'][seed] for seed in hardness_table[label]['Miss']])
        sorted_misses = np.sort(miss_scores)
        cumulative_misses = np.arange(1, len(sorted_misses) + 1) / len(sorted_misses)
        plt.plot(sorted_misses, cumulative_misses, label=f"{label} - Miss", linestyle='--')

    # Configurações do gráfico
    plt.title('Cumulative Hardness Histogram')
    plt.xlabel('Hardness Score')
    plt.ylabel('Cumulative Probability')
    plt.legend()
    plt.grid(True)
    plt.show()

# Exibir o gráfico
plot_cumulative_histogram(hardness_table)


In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Função para criar o gráfico cumulativo para uma seed específica
def plot_cumulative_histogram_single_seed(hardness_table, seed):
    plt.figure(figsize=(10, 6))

    for label in ['label_0', 'label_1']:
        # Hits para a seed especificada
        hit_scores = hardness_table[label]['Hit'][seed]
        sorted_hits = np.sort(hit_scores)
        cumulative_hits = np.arange(1, len(sorted_hits) + 1) / len(sorted_hits)
        plt.plot(sorted_hits, cumulative_hits, label=f"{label} - Hit")

        # Misses para a seed especificada
        miss_scores = hardness_table[label]['Miss'][seed]
        sorted_misses = np.sort(miss_scores)
        cumulative_misses = np.arange(1, len(sorted_misses) + 1) / len(sorted_misses)
        plt.plot(sorted_misses, cumulative_misses, label=f"{label} - Miss", linestyle='--')

    # Configurações do gráfico
    plt.title(f'Cumulative Hardness Histogram for {seed}')
    plt.xlabel('Hardness Score')
    plt.ylabel('Cumulative Probability')
    plt.legend()
    plt.grid(True)
    plt.show()

# Exibir o gráfico para uma seed específica
for i in range(30):
    plot_cumulative_histogram_single_seed(hardness_table, f'seed-{i}')
